# 1141ML Final - DSM 2D - Small Pic

In [ ]:
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import torch.nn as nn


transform = transforms.Compose([
    transforms.ToTensor(),          # shape: (1, 28, 28)
    transforms.Lambda(lambda x: x*2 - 1)  # scale to [-1, 1]
])

dataset = datasets.MNIST(root="./mnist_data", train=True,
                         download=True, transform=transform)

dataloader = DataLoader(dataset, batch_size=128, shuffle=True)


Failed to download (trying next):
HTTP Error 404: Not Found



100.0%


Extracting ./mnist_data\MNIST\raw\train-images-idx3-ubyte.gz to ./mnist_data\MNIST\raw

Failed to download (trying next):
HTTP Error 404: Not Found



100.0%


Extracting ./mnist_data\MNIST\raw\train-labels-idx1-ubyte.gz to ./mnist_data\MNIST\raw

Failed to download (trying next):
HTTP Error 404: Not Found



100.0%


Extracting ./mnist_data\MNIST\raw\t10k-images-idx3-ubyte.gz to ./mnist_data\MNIST\raw

Failed to download (trying next):
HTTP Error 404: Not Found



100.0%

Extracting ./mnist_data\MNIST\raw\t10k-labels-idx1-ubyte.gz to ./mnist_data\MNIST\raw



In [ ]:
class ScoreNet(nn.Module):
    def __init__(self, sigma):
        super().__init__()
        self.sigma = sigma
        
        self.net = nn.Sequential(
            nn.Conv2d(1, 32, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 32, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 1, 3, padding=1),
        )

    def forward(self, x):
        return self.net(x)
